In [95]:
from bs4 import BeautifulSoup
import requests

In [96]:
url = 'https://pokemondb.net/pokedex/all'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [97]:
# Checks website html
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [98]:
f'''
    Finds table with the class name "data-table sticky-header block-wide".
    To check all the tables in a page, use this: {soup.find_all('table')}
'''

table = soup.find('table', class_ = "data-table sticky-header block-wide")

In [99]:
'''
    Finds all th (headers) from the table.
'''

titles = table.find_all('th')

In [100]:
'''
    Create list for title headers with list comprehensions.
    .text: removes <th> </th>
    .strip(): removes \n
'''

table_title_list = [title.text.strip() for title in titles]
print(table_title_list)

['#', 'Name', 'Type', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']


In [101]:
import pandas as pd

In [102]:
# store headers in Data Frame
df = pd.DataFrame(columns= table_title_list)

# Print out DataFrane content
df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed


In [103]:
'''
    Finds all row in the table.
'''

column_data = table.find_all('tr')
print(type(column_data))

<class 'bs4.element.ResultSet'>


In [104]:
'''
    Loops through a row and get data.
'''

for row in column_data[1:]: # starts at index 1 to exclude header row
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]

    length = len(df)    # stores how many rows in the df
    df.loc[length] = individual_row_data    # appends individual row data to df every time df updates length

In [105]:
# show the update DataFrame

df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,0002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,0003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,0003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1210,1023,Iron Crown,Steel Psychic,590,90,72,100,122,108,98
1211,1024,Terapagos Normal Form,Normal,450,90,65,85,65,85,60
1212,1024,Terapagos Terastal Form,Normal,600,95,95,110,105,110,85
1213,1024,Terapagos Stellar Form,Normal,700,160,105,110,130,110,85


In [106]:
'''
    Had to have a separate file path because Windows is protecting my folder for my GitHub Repo,
    thus, executable files cannot create changes.
'''
filepath = r'C:\Users\SERVIN\Desktop\Important local files\Data Analyst\scraped_data'

df.to_csv(f'{filepath}\poke_index_data.csv', index = False)

In [107]:
import sqlite3
import os
import plotly.express as px

In [108]:
df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,0002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,0003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,0003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1210,1023,Iron Crown,Steel Psychic,590,90,72,100,122,108,98
1211,1024,Terapagos Normal Form,Normal,450,90,65,85,65,85,60
1212,1024,Terapagos Terastal Form,Normal,600,95,95,110,105,110,85
1213,1024,Terapagos Stellar Form,Normal,700,160,105,110,130,110,85


In [109]:
'''
    I already have instance of the df so I am commenting out this file line of code for now.
'''
# df = pd.read_csv('datasets\poke_index_data.csv')  # load csv file

'\n    I already have instance of the df so I am commenting out this file line of code for now.\n'

In [110]:
# Creating SQLite database
db_filepath = r'C:\Users\SERVIN\Desktop\Important local files\Data Analyst\scraped_data'

conn = sqlite3.connect(f'{db_filepath}\database.db')
cursor = conn.cursor()

In [120]:
create_table = '''CREATE TABLE IF NOT EXISTS pokemonData(
'#' INTEGER NOT NULL,
Name VARCHAR(40) NOT NULL,
Type VARCHAR(40) NOT NULL,
Total INTEGER NOT NULL,
HP INTEGER NOT NULL,
Attack INTEGER NOT NULL,
Defense INTEGER NOT NULL,
'Sp. Atk' INTEGER NOT NULL,
'Sp. Def' INTEGER NOT NULL,
Speed INTEGER NOT NULL);
'''
cursor.execute(create_table)    # exectue sql command   
conn.commit()   # commit sql command

In [119]:
# # Drop the table if it exists
# cursor.execute("DROP TABLE IF EXISTS pokemonData")

# # Commit the changes (optional for DROP TABLE)
# conn.commit()

In [121]:
'''
    Loading df to SQLite table.
'''
df.to_sql('pokemonData', conn, if_exists='append', index = False)



1215

In [122]:
# Verify the data types in the table
cursor.execute("PRAGMA table_info(pokemonData)")
table_info = cursor.fetchall()
for column in table_info:
    print(column)

(0, '#', 'INTEGER', 1, None, 0)
(1, 'Name', 'VARCHAR(40)', 1, None, 0)
(2, 'Type', 'VARCHAR(40)', 1, None, 0)
(3, 'Total', 'INTEGER', 1, None, 0)
(4, 'HP', 'INTEGER', 1, None, 0)
(5, 'Attack', 'INTEGER', 1, None, 0)
(6, 'Defense', 'INTEGER', 1, None, 0)
(7, 'Sp. Atk', 'INTEGER', 1, None, 0)
(8, 'Sp. Def', 'INTEGER', 1, None, 0)
(9, 'Speed', 'INTEGER', 1, None, 0)


In [ ]:
'''
    Read sql with pd.
'''
pd.read_sql('''SELECT * FROM pokemonData''', conn)

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,0002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,0003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,0003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1210,1023,Iron Crown,Steel Psychic,590,90,72,100,122,108,98
1211,1024,Terapagos Normal Form,Normal,450,90,65,85,65,85,60
1212,1024,Terapagos Terastal Form,Normal,600,95,95,110,105,110,85
1213,1024,Terapagos Stellar Form,Normal,700,160,105,110,130,110,85


In [123]:
conn.close()